In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
has_cv = True
split_size = 0.1 if has_cv else 1.0 

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
train.shape

In [ ]:
train.head()

In [ ]:
train_input_full = train.drop('label', axis=1) / 255.0
train_label_full = train['label'].to_numpy()
print(train_input_full.shape,train_label_full.shape)

In [ ]:
# Resize the image
train_input_numpy = train_input_full.to_numpy().reshape(train_input_full.shape[0], 28, 28 ,1)

In [ ]:
split = int(train_input_numpy.shape[0]*split_size)
# Train split
if (has_cv) :
    train_input, cv_input, train_label, cv_label = train_test_split(train_input_numpy, train_label_full, test_size = split_size, random_state=2)
else:
    train_input = train_input_numpy
    train_label = train_label_full
    

print(f'Train Input Shape : {train_input.shape}')
print(f'Train Label Shape : {train_label.shape}')
if (has_cv) :
    print(f'CV Input Shape : {cv_input.shape}')
    print(f'CV label Shape : {cv_label.shape}')

In [ ]:
tf.random.set_seed(1234)
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (5,5), padding = 'Same', activation="relu", input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, (5,5), padding = 'Same', activation="relu"),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3,3), activation="relu", padding = 'Same'),
    tf.keras.layers.Conv2D(64, (3,3), activation="relu", padding = 'Same'),
    tf.keras.layers.MaxPooling2D((2,2),strides=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation="sigmoid")
])

# model = tf.keras.models.Sequential()

# model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
#                  activation ='relu', input_shape = (28,28,1)))
# model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
#                  activation ='relu'))
# model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
# model.add(tf.keras.layers.Dropout(0.25))


# model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu'))
# model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu'))
# model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model.add(tf.keras.layers.Dropout(0.25))


# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(256, activation = "relu"))
# model.add(tf.keras.layers.Dropout(0.5))
# model.add(tf.keras.layers.Dense(10, activation = "softmax"))
model.summary()

In [ ]:
# data agumentaion
epochs = 30
# train_cat_label = tf.keras.utils.to_categorical(train_label, 10)
# if (has_cv):
#     cv_cat_label = tf.keras.utils.to_categorical(cv_label, 10)
    
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# learning rate decay after each epoch
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
if (has_cv):
    history = model.fit_generator(datagen.flow(train_input, train_label, batch_size=32),
                    steps_per_epoch=len(train_input)/32, epochs=epochs,
                    validation_data=(cv_input, cv_label), callbacks=[learning_rate_reduction])
else:
    history = model.fit_generator(datagen.flow(train_input, train_label, batch_size=32),
                    steps_per_epoch=len(train_input)/32, epochs=epochs, callbacks=[learning_rate_reduction])


In [ ]:
# file_path = "model.h5"
# checkpoint = tf.keras.callbacks.ModelCheckpoint(file_path, mointer="accuracy", verbose = 1, save_best_only = True, mode = 'min')

In [ ]:
# model.fit(cv_input, cv_label, epochs=3)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
if (has_cv):
    history = model.fit(train_input, train_label, epochs=30, validation_data=(cv_input, cv_label), verbose=1)
else:
    history = model.fit(train_input, train_label, epochs=50)

In [ ]:
acc = np.mean(history.history['accuracy'])
val_acc = np.mean(history.history['val_accuracy'])
print(acc, val_acc)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))

plt.plot(acc)
plt.plot(val_acc)
plt.show()

In [ ]:
# save_model = tf.keras.models.load_model(file_path)
# np.testing.assert_allclose(save_model.predict(train_input), model.predict(train_input), 1e-5)

In [ ]:
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
test_input_full = test / 255.0
test_input_numpy = test_input_full.to_numpy().reshape(test_input_full.shape[0], 28, 28 ,1)
test_input_numpy.shape

In [ ]:
predictions = model.predict(test_input_numpy)
preds = [np.argmax(x) for x in predictions]

In [ ]:
len = test_input_numpy.shape[0]
ids = np.array(range(1, len+1))
output = pd.DataFrame({'ImageId': ids, 'Label':preds})
output.to_csv('my_submission.csv', index=False)

In [ ]:
expected_cv  = model.predict(cv_input)
expected_cv = [np.argmax(x) for x in expected_cv]
check = pd.DataFrame({'predicted': expected_cv, 'original': cv_label})
check['boolval'] = check.predicted != check.original

In [ ]:
# check[['original', 'boolval']].groupby('original').sum()
pd.crosstab(check['original'], check['predicted'], margins=True)